In [1]:
import sqlite3
import matplotlib.pyplot as plt
import pandas as pd
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 700)

In [2]:
cnx = sqlite3.connect('Database/Civ6CitySettledData.db')
cur = cnx.cursor()
print(cnx)
print(cur)

## Retrieve the city plots as settled data
cityPlotsSettled contains the city plot, adjacent to city plots, and 2 tiles from city plots information at the time the city was settled. The idea is to look at this information and "predict" the yield performance of the city.

The base identiy for this database is the cityId. That is, in this database (and experiment) is designed to look at city growth over time. The same cityName can occur in multiple games so when we add the recorded information for the game we create new cityIds.

In [3]:
sqlSelect = 'SELECT * FROM cityPlotsSettled'
#sqlSelect = 'SELECT * FROM cityPlotsSettled WHERE ownerCityId <> "None"'
#sqlSelect = 'SELECT * FROM cityPlotsSettled WHERE ring == 0'
#sqlSelect = 'SELECT * FROM cityPlotsSettled WHERE ring == 1'
#sqlSelect = 'SELECT * FROM cityPlotsSettled WHERE ring == 2'
cityPs = pd.read_sql_query(sqlSelect, cnx)
print(cityPs.shape)
print(cityPs.dtypes)

(2432, 15)
plotId              int64
ownerCityId       float64
recordedCityId      int64
ring                int64
terrain            object
feature            object
resource           object
resourceCount       int64
resourceType       object
workers             int64
district           object
hasRiver            int64
isWater             int64
isLake              int64
isCity              int64
dtype: object


In [4]:
print(cityPs[['plotId', 'terrain']].groupby('terrain').count().sort_values(by='plotId', ascending=False))

                      plotId
terrain                     
Grassland                613
Plains                   595
Grassland (Hills)        287
Coast and Lake           266
Plains (Hills)           259
Desert                   138
Grassland (Mountain)      65
Desert (Hills)            53
Plains (Mountain)         44
Tundra                    44
Ocean                     30
Desert (Mountain)         18
Tundra (Hills)            15
Tundra (Mountain)          4
Snow                       1


In [6]:
print(cityPs[['plotId', 'feature']].groupby('feature').count().sort_values(by='plotId', ascending=False))

             plotId
feature            
None           1793
Woods           333
Rainforest      206
Marsh            54
Floodplains      31
Reef             12
Oasis             3


In [7]:
print(cityPs[['plotId', 'resource']].groupby('resource').count().sort_values(by='plotId', ascending=False))
print(len(cityPs[['plotId', 'resource']].groupby('resource').count().sort_values(by='plotId', ascending=False)))

          plotId
resource        
None        1902
Stone         59
Wheat         57
Rice          33
Cattle        31
Fish          27
Bananas       24
Sheep         24
Horses        22
Iron          18
Coal          17
Deer          16
Aluminum      15
Uranium       13
Copper        12
Niter         12
Crabs         10
Cotton        10
Gypsum        10
Jade           8
Wine           8
Furs           8
Mercury        7
Turtles        7
Ivory          7
Silk           6
Oil            6
Truffles       6
Tobacco        6
Diamonds       6
Citrus         5
Marble         5
Dyes           5
Pearls         5
Spices         4
Incense        4
Salt           3
Cocoa          3
Coffee         2
Sugar          2
Tea            2
Olives         2
Whales         2
Silver         1
44


In [7]:
#print(cityPs[['plotId', 'resourceType']].groupby('resourceType').count().sort_values(by='plotId', ascending=False))

In [8]:
#print(cityPs[['plotId', 'district']].groupby('district').count().sort_values(by='plotId', ascending=False))

In [9]:
print(cityPs[['plotId', 'hasRiver']].groupby('hasRiver').count().sort_values(by='plotId', ascending=False))

In [10]:
#print(cityPs[['plotId', 'resourceCount']].groupby('resourceCount').count().sort_values(by='plotId', ascending=False))

In [12]:
#cityPs[cityPs['resourceType'] == 'Strategic'].resource.unique()
#cityPs[cityPs['resourceType'] == 'Luxury'].resource.unique()
#cityPs[cityPs['resourceType'] == 'Bonus'].resource.unique()

In [13]:
#cols = list(cityPs['category'].unique())
#print(type(cols))
#print(len(cols))
#print(cols)

In [18]:
cur.close()
cnx.close()